In [68]:
# Předpoklady:
    # Uživatel nakreslí jednoduchý mnohoúhelník
    # Uživatel ho nakreslí s pořadím bodů orientovaným proti směru hodinových ručiček

In [66]:
import tkinter as tk
from tkinter import messagebox
from shapely.geometry import LineString
import numpy as np


In [62]:
# setup the canvas

window = tk.Tk()
window.title("Computational Geometry")

canvas_width = 400
canvas_height = 400

canvas = tk.Canvas(window, width=canvas_width, height=canvas_height)
canvas.pack()

In [63]:
# set up points storage and drawing polygons
points = []
last_points = []

def on_canvas_click(click):
    x = click.x
    y = click.y
    points.append((x, y))
    canvas.create_oval(x-2, y-2, x+2, y+2, fill="black")  # show the clicked point on the canvas
    
    # lines between the last two points
    if len(points) >= 2:
        last_points = points[-2:]
        canvas.create_line(last_points, fill="black")

canvas.bind("<Button-1>", on_canvas_click) # Button-1 is the left-click

'2789352931528on_canvas_click'

In [ ]:
# triangulation

def check_intersection(first1, first2, second1, second2):
    first_line = LineString([first1, first2])
    second_line = LineString([second1, second2])
    if first_line.intersects(second_line):
        return True
    return False

def is_on_the_right(start, end, checked): # check using vectors and cross product
    v1 = (end[0] - start[0], end[1] - start[1]) 
    v2 = (checked[0] - start[0], checked[1] - start[1])
    cross_product = v1[0] * v2[1] - v1[1] * v2[0]
    if cross_product < 0:
        return True
    return False

def sum_triangles(triangles):
    total_area = 0
    for triangle in triangles:
        area = triangle_area(triangle)
        total_area += area
    return total_area

def triangle_area(triangle):
    x1, y1 = triangle[0]
    x2, y2 = triangle[1]
    x3, y3 = triangle[2]
    area = 0.5 * abs((x1 * (y2 - y3) + x2 * (y3 - y1) + x3 * (y1 - y2))) # shoelace formula
    return area

def triangulate_polygon(points):
    while not points: # until it is empty
        

In [64]:
# buttons

def calculate_area():
    canvas.unbind("<Button-1>") # stops drawing
    calculate_button.config(state=tk.DISABLED)
    # TODO: calculation

def close_application():
    window.destroy()
    
def clear_canvas():
    canvas.delete("all")
    points.clear()
    canvas.bind("<Button-1>", on_canvas_click)
    calculate_button.config(state=tk.NORMAL)

calculate_button = tk.Button(window, text="Calculate area", command=calculate_area)
calculate_button.pack(side=tk.LEFT, padx=5)

clear_button = tk.Button(window, text="Clear", command=clear_canvas)
clear_button.pack(side=tk.LEFT, padx=5)

close_button = tk.Button(window, text="Close", command=close_application)
close_button.pack(side=tk.LEFT, padx=5)

In [65]:
window.mainloop()

In [47]:
points

[(179, 310), (125, 241), (309, 162), (259, 242)]